### RQ2: What is the behavior of blood donation organizations on social media?
1. Who are the most active blood donating organizations in Milan and NYC on Twitter and Instagram? 
2. How has their influence changed over time?
3. Do blood donation organizations in both locations behave similarly on social media ie: frequency of posts, number of followers, etc.?
     - what are the most active posts for these organizations in terms of posts and mentions?
     - Are they are any similarities/differences?

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import datetime as dt

In [2]:
#load data
inst_ig = pd.read_csv('Instagram/institution_raw.csv')
groupedInst_ig = pd.read_csv('Instagram/groupedInstitutions.csv')
mentions_ig = pd.read_csv('Instagram/mentioned_raw.csv')
groupedMentions_ig = pd.read_csv('Instagram/mentioned.csv')
inst_tw = pd.read_csv('Twitter/institution_raw_tw.csv')
groupedInst_tw = pd.read_csv('Twitter/groupedInstitutions_tw.csv')
mentions_tw = pd.read_csv('Twitter/mentioned_raw_tw.csv')
groupedMentions_tw = pd.read_csv('Twitter/mentioned_tw.csv')

In [3]:
inst_ig.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'conversation_id', 'created_at',
       'date', 'time', 'timezone', 'user_id', 'username', 'name', 'place',
       'lat', 'lng', 'tweet', 'urls', 'hashtags', 'language', 'mentions',
       'photos', 'replies_count', 'retweets_count', 'likes_count', 'cashtags',
       'link', 'retweet', 'quote_url', 'video', 'thumbnail', 'sentiment'],
      dtype='object')

Note: the institution data frames contain posts posted by NYBC, AVIS and FIDAS. The mentions data frames contain posts that mention these organizations.

#### Subquestion 1: Who are the most active blood donation organizations in NYC and Milan on Instagram and Twitter? <br>

In [4]:
#function to stack Twitter and Instagram dfs in case we want to visualize together in RAWgraphs
def stack_dfs(ig_df,tw_df):
    ig_df['platform'] = 'Instagram'
    tw_df['platform'] = 'Twitter'
    df_all = ig_df.append(tw_df).reset_index(drop=True)
    #standardize usernames to compare across platforms
    df_all = df_all.replace('newyorkbloodcenter','nybloodcenter')
    df_all = df_all.replace('fidasdonatorisangue','fidasnazionale') #confirm this is the same
    return df_all

In [5]:
groupedInstitutions_all = stack_dfs(groupedInst_ig,groupedInst_tw)
#export - visualize in RAWgraphs as bubble graphs
groupedInstitutions_all.to_csv('data_rawgraphs/groupedInstitutions_all.csv')
groupedInstitutions_all

,username,likes_count,retweet,sentiment,n_post,followers,platform
0,avisnazionale,596.725000,0.025000,-0.015833,40,29000,Instagram
1,fidasnazionale,57.000000,0.444444,0.000000,9,5624,Instagram
2,nybloodcenter,65.851504,0.112782,0.226000,532,3463,Instagram
3,avis_milano,0.333333,0.000000,0.011601,51,6460,Twitter
4,avisnazionale,7.870907,0.000000,0.004783,3207,580,Twitter
5,fidasnazionale,2.698595,0.000000,-0.000180,783,1401,Twitter
6,nybloodcenter,2.791457,0.000000,0.287385,398,4934,Twitter
7,redcrosseny,1.419355,0.000000,0.321224,31,3158,Twitter


In [6]:
#do the same for the grouped mentions dfs
groupedMentions_all = stack_dfs(groupedMentions_ig,groupedMentions_tw)
#export - visualize in RAWgraphs as bubble graphs
groupedMentions_all.to_csv('data_rawgraphs/groupedMentions_all.csv')
groupedMentions_all

,cited,likes_count,retweet,sentiment,platform
0,avisnazionale,44.359039,0.006321,0.036417,Instagram
1,fidasnazionale,31.297872,0.000000,0.000000,Instagram
2,nybloodcenter,68.438356,0.054795,0.173053,Instagram
3,AVIS_Milano,2.772727,0.000000,0.028409,Twitter
4,FIDASnazionale,3.436620,0.000000,0.001568,Twitter
5,NYBloodCenter,2.101325,0.000000,0.084389,Twitter
6,RedCrossENY,2.962963,0.000000,0.175902,Twitter
7,avisnazionale,2.314533,0.000000,0.011824,Twitter


Based on followers, avisnazionale has the biggest following on Instagram. However, even though they have less of a following on Twitter, that is the org and platform that posts the most. 

In [7]:
#create dfs to plot monthly posts by and mentions of org
def monthly_groupby_posts(data):
    df = data.copy()
    df['Number of Posts'] = 1
    df['date'] = pd.to_datetime(df['date'])
    df['year-month'] = df['date'].dt.strftime('%Y-%m')
    df = df.groupby(['year-month','username']).agg({'Number of Posts':sum}).reset_index(drop=False)
    return df

def monthly_groupby_mentions(data):
    df = data.copy()
    df['Number of Mentions'] = 1
    df['date'] = pd.to_datetime(df['date'])
    df['year-month'] = df['date'].dt.strftime('%Y-%m')
    df = df.groupby(['year-month','cited']).agg({'Number of Mentions':sum}).reset_index(drop=False)
    return df

In [8]:
#apply function and export to visualize in RAWgraphs as stream graphs
monthly_posts_ig = monthly_groupby_posts(inst_ig)
monthly_posts_ig.to_csv('data_rawgraphs/monthly_posts_ig.csv')
monthly_posts_tw = monthly_groupby_posts(inst_tw)
monthly_posts_tw.to_csv('data_rawgraphs/monthly_posts_tw.csv')
monthly_mentions_ig = monthly_groupby_mentions(mentions_ig)
monthly_mentions_ig.to_csv('data_rawgraphs/monthly_mentions_ig.csv')
monthly_mentions_tw = monthly_groupby_mentions(mentions_tw)
monthly_mentions_tw.to_csv('data_rawgraphs/monthly_mentions_tw.csv')

In [9]:
#check the top 5 months for posts and mentions for each org 
def top5_months_posts(data,org):
    #filter by org
    data = data.loc[data['username'] == org]
    top5 = data.sort_values(by=['Number of Posts'],ascending=False)[:5]
    return top5

def top5_months_mentions(data,org):
    #filter by org
    data = data.loc[data['cited'] == org]
    top5 = data.sort_values(by=['Number of Mentions'],ascending=False)[:5]
    return top5

In [10]:
orgs_ig = list(inst_ig.username.unique())
top5_months_posts_ig = []
for org in orgs_ig:
    top5_months_posts_ig.append(top5_months_posts(monthly_posts_ig,org))
#display
for i in top5_months_posts_ig:
    display(i)

,year-month,username,Number of Posts
32,2020-12,newyorkbloodcenter,55
28,2020-10,newyorkbloodcenter,42
36,2021-02,newyorkbloodcenter,37
27,2020-09,newyorkbloodcenter,34
26,2020-08,newyorkbloodcenter,31


,year-month,username,Number of Posts
18,2020-04,avisnazionale,19
16,2020-03,avisnazionale,15
21,2020-05,avisnazionale,3
29,2020-11,avisnazionale,3


,year-month,username,Number of Posts
31,2020-12,fidasdonatorisangue,2
12,2020-01,fidasdonatorisangue,1
14,2020-02,fidasdonatorisangue,1
19,2020-04,fidasdonatorisangue,1
22,2020-05,fidasdonatorisangue,1


In [11]:
orgs_tw = list(inst_tw.username.unique())
top5_months_posts_tw = []
for org in orgs_tw:
    top5_months_posts_tw.append(top5_months_posts(monthly_posts_tw,org))
#display
for i in top5_months_posts_tw:
    display(i)

,year-month,username,Number of Posts
7,2014-06,avis_milano,5
12,2014-12,avis_milano,4
4,2014-03,avis_milano,4
2,2014-01,avis_milano,3
3,2014-02,avis_milano,3


,year-month,username,Number of Posts
33,2015-05,fidasnazionale,40
217,2020-03,fidasnazionale,36
259,2021-03,fidasnazionale,28
103,2017-03,fidasnazionale,21
221,2020-04,fidasnazionale,20


,year-month,username,Number of Posts
72,2016-05,avisnazionale,121
108,2017-05,avisnazionale,117
49,2015-10,avisnazionale,104
111,2017-06,avisnazionale,95
37,2015-06,avisnazionale,83


,year-month,username,Number of Posts
113,2017-06,nybloodcenter,23
154,2018-06,nybloodcenter,22
17,2015-01,nybloodcenter,19
25,2015-03,nybloodcenter,17
30,2015-04,nybloodcenter,16


,year-month,username,Number of Posts
219,2020-03,redcrosseny,4
257,2021-02,redcrosseny,4
253,2021-01,redcrosseny,4
61,2016-01,redcrosseny,3
193,2019-07,redcrosseny,2


In [12]:
orgs_ig_mentions = list(mentions_ig.cited.unique())
top5_months_mentions_ig = []
for org in orgs_ig_mentions:
    top5_months_mentions_ig.append(top5_months_mentions(monthly_mentions_ig,org))
#display
for i in top5_months_mentions_ig:
    display(i)

,year-month,cited,Number of Mentions
29,2020-03,avisnazionale,223
32,2020-04,avisnazionale,202
35,2020-05,avisnazionale,64
58,2021-02,avisnazionale,58
55,2021-01,avisnazionale,53


,year-month,cited,Number of Mentions
59,2021-02,fidasdonatorisangue,12
56,2021-01,fidasdonatorisangue,9
30,2020-03,fidasdonatorisangue,8
62,2021-03,fidasdonatorisangue,6
33,2020-04,fidasdonatorisangue,5


,year-month,cited,Number of Mentions
54,2020-12,newyorkbloodcenter,144
57,2021-01,newyorkbloodcenter,113
48,2020-10,newyorkbloodcenter,93
46,2020-09,newyorkbloodcenter,89
51,2020-11,newyorkbloodcenter,81


In [13]:
orgs_tw_mentions = list(mentions_tw.cited.unique())
top5_months_mentions_tw = []
for org in orgs_tw_mentions:
    top5_months_mentions_tw.append(top5_months_mentions(monthly_mentions_tw,org))
#display
for i in top5_months_mentions_tw:
    display(i)

,year-month,cited,Number of Mentions
186,2019-12,avisnazionale,793
127,2018-07,avisnazionale,771
196,2020-03,avisnazionale,706
117,2018-04,avisnazionale,415
13,2015-06,avisnazionale,216


,year-month,cited,Number of Mentions
235,2021-03,AVIS_Milano,2
204,2020-06,AVIS_Milano,2
192,2020-03,AVIS_Milano,2
181,2019-11,AVIS_Milano,2
16,2015-08,AVIS_Milano,1


,year-month,cited,Number of Mentions
20,2015-09,NYBloodCenter,140
17,2015-08,NYBloodCenter,86
226,2020-12,NYBloodCenter,70
12,2015-06,NYBloodCenter,57
194,2020-03,NYBloodCenter,41


,year-month,cited,Number of Mentions
81,2017-06,FIDASnazionale,7
34,2016-02,FIDASnazionale,6
165,2019-06,FIDASnazionale,5
193,2020-03,FIDASnazionale,5
39,2016-04,FIDASnazionale,5


,year-month,cited,Number of Mentions


,year-month,cited,Number of Mentions
109,2018-02,RedCrossENY,3
122,2018-06,RedCrossENY,3
195,2020-03,RedCrossENY,3
147,2019-01,RedCrossENY,2
179,2019-10,RedCrossENY,2


In [14]:
#check time frame of each platform
print('The time frame for Twitter mentions is {0} to {1}'.format(mentions_tw.date.min(),mentions_tw.date.max()))
print('The time frame for Instagram mentions is {0} to {1}'.format(mentions_ig.date.min(),mentions_ig.date.max()))
print('The time frame for Twitter posts is {0} to {1}'.format(inst_tw.date.min(),inst_tw.date.max()))
print('The time frame for Instagram posts is {0} to {1}'.format(inst_ig.date.min(),inst_ig.date.max()))

The time frame for Twitter mentions is 2014-12-22 to 2021-03-16
The time frame for Instagram mentions is 2018-03-22 00:00:00 to 2021-03-29 00:00:00
The time frame for Twitter posts is 2013-10-23 to 2021-03-16
The time frame for Instagram posts is 2019-01-04 00:00:00 to 2021-03-29 00:00:00


In [16]:
#check the number of mentions for each org and platform - any differences?
def mentions_count(data):
    mentions_total = data.copy()
    mentions_total['date'] = pd.to_datetime(mentions_total['date'])
    mentions_total = mentions_total.loc[(mentions_total['date'] >= '2018-03-22') &
                                       (mentions_total['date'] <= '2021-03-16')]
    mentions_total = mentions_total[['cited','id']].groupby(['cited']).count().rename(columns={'id':'Mentions'}).reset_index()
    #standardize usernames to compare across platforms
    mentions_total = mentions_total.replace('newyorkbloodcenter','NYBloodCenter')
    mentions_total = mentions_total.replace('fidasdonatorisangue','FIDASnazionale') #confirm this is the same
    return mentions_total

In [17]:
#export to visualize in RAWgraphs
total_mentions_ig = mentions_count(mentions_ig)
total_mentions_ig.to_csv('data_rawgraphs/total_mentions_ig.csv')
total_mentions_tw = mentions_count(mentions_tw)
total_mentions_tw.to_csv('data_rawgraphs/total_mentions_tw.csv')
# stack df for comparison
total_mentions_all = stack_dfs(total_mentions_ig,total_mentions_tw)
total_mentions_all.to_csv('data_rawgraphs/total_mentions_all.csv')
#or merge togther on cited 
total_mentions_all2 = pd.merge(total_mentions_tw,total_mentions_ig,
                        on='cited',how='left').fillna(0).rename(columns= {'Mentions_x':'Twitter','Mentions_y':'Instagram'})
total_mentions_all2 = total_mentions_all2[['cited','Twitter','Instagram']]

In [18]:
total_mentions_all2

,cited,Twitter,Instagram
0,AVIS_Milano,16,0.0
1,FIDASnazionale,81,46.0
2,NYBloodCenter,589,1437.0
3,RedCrossENY,19,0.0
4,avisnazionale,6039,784.0


#### Subquestion 2: How has the activity of these organizations changed over time?
1. Compare the engagement metrics (replies_count,likes_count,retweets_count) for each post posted by these organizations and mentioning these organizations over time. 
2. Identify the most relevant posts based on these metrics for each org.
3. Is there any relationship between the engagment metrics and sentiment?

#### 2.1.1: Compare engagment metrics over time for posts posted by blood donating organizations

In [19]:
#check columns for engagement metrics
inst_ig.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'conversation_id', 'created_at',
       'date', 'time', 'timezone', 'user_id', 'username', 'name', 'place',
       'lat', 'lng', 'tweet', 'urls', 'hashtags', 'language', 'mentions',
       'photos', 'replies_count', 'retweets_count', 'likes_count', 'cashtags',
       'link', 'retweet', 'quote_url', 'video', 'thumbnail', 'sentiment'],
      dtype='object')

In [20]:
#create dfs with just engagement metrics
metrics = ['date','username','tweet','replies_count','retweets_count','likes_count']
inst_ig_metrics = inst_ig[metrics]
inst_ig_metrics['date'] = pd.to_datetime(inst_ig_metrics['date'])

inst_tw_metrics = inst_tw[metrics]
inst_tw_metrics['date'] = pd.to_datetime(inst_tw_metrics['date'])

#stack ig and tw dfs
inst_all_metrics = stack_dfs(inst_ig_metrics,inst_tw_metrics)

#export to visualize in raw graphs as beeswarm plot
inst_ig_metrics.to_csv('data_rawgraphs/org_ig_metrics.csv')
inst_tw_metrics.to_csv('data_rawgraphs/org_tw_metrics.csv')
inst_all_metrics.to_csv('data_rawgraphs/org_all_metrics.csv')

<ipython-input-20-fc2eb5f6b8d3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inst_ig_metrics['date'] = pd.to_datetime(inst_ig_metrics['date'])
<ipython-input-20-fc2eb5f6b8d3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inst_tw_metrics['date'] = pd.to_datetime(inst_tw_metrics['date'])
<ipython-input-4-0d11d79ceb99>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

##### 2.1.2: Compare engagement metrics over time for posts mentioning blood donating organizations

In [21]:
#create dfs with just engagement and sentiment metrics(check to see if the tweets mentioning orgs are +/-)
metrics_mentions = ['date','username','tweet','replies_count','retweets_count','likes_count','cited','sentiment']
mentions_ig_metrics = mentions_ig[metrics_mentions]
mentions_ig_metrics['date'] = pd.to_datetime(mentions_ig_metrics['date'])

mentions_tw_metrics = mentions_tw[metrics_mentions]
mentions_tw_metrics = mentions_tw_metrics.dropna(subset=['cited'])

#stack ig and tw dfs
mentions_all_metrics = stack_dfs(mentions_ig_metrics,mentions_tw_metrics)

#export to visualize in raw graphs as beeswarm plot
mentions_ig_metrics.to_csv('data_rawgraphs/mentions_ig_metrics.csv')
mentions_tw_metrics.to_csv('data_rawgraphs/mentions_tw_metrics.csv')
mentions_all_metrics.to_csv('data_rawgraphs/mentions_all_metrics.csv')

<ipython-input-21-0551d601d595>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mentions_ig_metrics['date'] = pd.to_datetime(mentions_ig_metrics['date'])
<ipython-input-4-0d11d79ceb99>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ig_df['platform'] = 'Instagram'


#### 2.2.1 Which posts are mentioning these organizations are generating the most engagment?
Can we identify any similarities/differences between these posts?

In [22]:
#so we can see the entire tweet
pd.set_option("display.max_colwidth", -1)

def top5_posts(data,org,metric):
    top5 = data.loc[data['username']==org]
    top5 = top5[['date','username','tweet',metric,'sentiment']].sort_values(metric,ascending=False)
    top5 = top5[:5].reset_index()
    return top5

def top5_mentions(data,org,metric):
    top5 = data.loc[data['cited']==org]
    top5 = top5[['date','username','cited','tweet',metric,'sentiment']].sort_values(metric,ascending=False)
    top5 = top5[:5].reset_index()
    return top5

<ipython-input-22-e6b314f2aaf8>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [23]:
#top 5 posts - Instagram
orgs_ig = list(inst_ig.username.unique())
metric_ig = ['likes_count','replies_count']
top5_posts_ig = []
for org in orgs_ig:
    for metric in metric_ig:
        top5_posts_ig.append(top5_posts(inst_ig,org,metric))

In [24]:
for i in top5_posts_ig:
    display(i)

,index,date,username,tweet,likes_count,sentiment
0,541,2019-09-03 00:00:00,newyorkbloodcenter,"A HUGE congratulations to Michael St. Amour! He recently made his 400th platelet donation at Grand Central Donor Center. We are so thankful for Michael, and dedicated donors just like him who continue to save lives with us! #nybloodcenter #platelets #newyorkbloodcenter",289,0.175000
1,540,2020-04-06 00:00:00,newyorkbloodcenter,"We have something very exciting to share: Ms. Madalyn Connors has the distinction of being the first convalescent plasma donor on Long Island! Let's show Madalyn just how proud and thankful we are for her dedication to helping others! We are so very excited to be able to make a big impact on our communities at this time, and we are incredibly grateful for donors like Madalyn who have stepped up to be part of this effort! #nybloodcenter\n.\n.\n.\n #newyorkbloodcenter #donateblood #blooddonor #nyc #nyclife #newyork #ny #newyorker #newyorklife #newyorklike #blooddonation #giveblood #blooddonors #donatingblood #iloveny #ilovenewyork #adrp #coronavirus #covid19 #covid_19 #covid-19 #convalescentplasma #givebloodchallenge #combatcoronavirus #combatcovid19 @adrpgiveblood #todaysheroes @todayshow #unsungheroes",285,0.445625
2,539,2020-06-27 00:00:00,newyorkbloodcenter,"To all of our class of 2020 donors, thank you stepping up to save lives. Your dedication to donating, whether at your school, a community blood drive, or a donor center, only further shows your commitment to helping others. We are proud to have some many youth donors, volunteers and even coordinators! We wish you the absolute best of luck in your future endeavors! #nybloodcenter\n.\n.\n.\n#newyorkbloodcenter #donateblood #blooddonor #nyc #nyclife #newyork #ny #newyorker #newyorklife #newyorklike #blooddonation #giveblood #blooddonors #donatingblood #iloveny #ilovenewyork #adrp #classof2020 #somegoodnews #givebloodchallenge",250,0.315625
3,538,2021-03-18 00:00:00,newyorkbloodcenter,"We need your help NOW. The blood supply is dangerously low and we need donors TODAY in order to support area hospitals. New York’s healthcare system requires 1,500 donations each day to treat patients ranging from trauma victims to newborn babies and their mothers to cancer patients. Save a life now - link in bio to make your appointment! #nybloodcenter\n.\n.\n.\n#newyorkbloodcenter #donateblood #blooddonor #nyc #nyclife #newyork #ny #newyorker #newyorklife #newyorklike #blooddonation #giveblood #blooddonors #donatingblood #iloveny #ilovenewyork #adrp #givebloodnyc",241,0.085227
4,537,2020-09-22 00:00:00,newyorkbloodcenter,"This might be the coolest #TattooTuesday we've ever had. Meet Giulia P. - blood donor, and owner of one epic tattoo. Giulia is O-, and although she hates needles, she got this tattoo as a reminder of just how important it is for her to give blood. O- is the universal donor, and ALWAYS needed. Shout out to Giulia P., and all of our dedicated donors who continue to serve the community by giving blood! #nybloodcenter \n.\n.\n.\n#newyorkbloodcenter #donateblood #blooddonor #nyc #nyclife #newyork #ny #newyorker #newyorklife #newyorklike #blooddonation #giveblood #blooddonors #donatingblood #iloveny #ilovenewyork #adrp @adrpgiveblood #tattoo #givebloodchallenge",235,0.166667


,index,date,username,tweet,replies_count,sentiment
0,541,2019-09-03 00:00:00,newyorkbloodcenter,"A HUGE congratulations to Michael St. Amour! He recently made his 400th platelet donation at Grand Central Donor Center. We are so thankful for Michael, and dedicated donors just like him who continue to save lives with us! #nybloodcenter #platelets #newyorkbloodcenter",17,0.175000
1,532,2020-03-19 00:00:00,newyorkbloodcenter,"We’ve increased capacity at our donor centers by extending hours and opening for extra days each week. These modifications will provide controlled, safe environments for healthy donors. By quickly implementing this sustainable long-term solution we have worked to prevent our blood supply from dropping to dangerously low levels.\n.\nMake your appointment to give at a donor center - link in bio #nybloodcenter\n.\n.\n.\n#newyorkbloodcenter #donateblood #blooddonor # nyc #nyclife #newyork #ny #newyorker #newyorklife #newyorklike #blooddonation #giveblood #blooddonors #donating blood #iloveny #ilovenewyork #adrp #covid19 #coronavirus",16,0.205556
2,540,2020-04-06 00:00:00,newyorkbloodcenter,"We have something very exciting to share: Ms. Madalyn Connors has the distinction of being the first convalescent plasma donor on Long Island! Let's show Madalyn just how proud and thankful we are for her dedication to helping others! We are so very excited to be able to make a big impact on our communities at this time, and we are incredibly grateful for donors like Madalyn who have stepped up to be part of this effort! #nybloodcenter\n.\n.\n.\n #newyorkbloodcenter #donateblood #blooddonor #nyc #nyclife #newyork #ny #newyorker #newyorklife #newyorklike #blooddonation #giveblood #blooddonors #donatingblood #iloveny #ilovenewyork #adrp #coronavirus #covid19 #covid_19 #covid-19 #convalescentplasma #givebloodchallenge #combatcoronavirus #combatcovid19 @adrpgiveblood #todaysheroes @todayshow #unsungheroes",15,0.445625
3,139,2021-01-08 00:00:00,newyorkbloodcenter,"Someone, right now, is waiting for a hero. Convalescent plasma is a type of donation given by those who have recovered from COVID-19 and contains antibodies that may help patients in need. Learn more and become a donor today - go to nybc.org/cpdonor\n.\n.\n.\n#nybloodcenter #newyorkbloodcenter #donateblood #blooddonor #nyc #nyclife #newyork #ny #newyorker #newyorklife #newyorklike #blooddonation #giveblood #blooddonors #donatingblood #iloveny #ilovenewyork #adrp #coronavirus #covid19 #covid_19 #givebloodchallenge #combatcoronavirus #combatcovid19 @adrpgiveblood #convalescentplasma #plasma #donateCOVIDplasma",15,0.392857
4,142,2021-01-08 00:00:00,newyorkbloodcenter,"Someone, right now, is waiting for a hero. Convalescent plasma is a type of donation given by those who have recovered from COVID-19 and contains antibodies that may help patients in need. Learn more and become a donor today - go to nybc.org/cpdonor\n.\n.\n.\n#nybloodcenter #newyorkbloodcenter #donateblood #blooddonor #nyc #nyclife #newyork #ny #newyorker #newyorklife #newyorklike #blooddonation #giveblood #blooddonors #donatingblood #iloveny #ilovenewyork #adrp #coronavirus #covid19 #covid_19 #givebloodchallenge #combatcoronavirus #combatcovid19 @adrpgiveblood #convalescentplasma #plasma #donateCOVIDplasma",15,0.392857


,index,date,username,tweet,likes_count,sentiment
0,580,2020-03-15 00:00:00,avisnazionale,"#Avis ha 3400 sedi sul territorio nazionale, tutte gestite da volontari (con l'aiuto, in quelle più grandi, di qualche dipendente). Persone che hanno deciso di dedicare ad AVIS parte del loro tempo libero anche al tempo del #coronavirus. A tutti loro va la nostra immensa gratitudine. Senza di loro Avis non esisterebbe.😍 #NoiSiamoAvis #escosoloperdonare #donazione #sangue #plasma #piastrine #ig_avis",1478,0.0
1,579,2020-04-25 00:00:00,avisnazionale,"🇮🇹 #25Aprile #AVIS\n""“Dietro ogni articolo della Costituzione, o giovani, voi dovete vedere giovani come voi che hanno dato la vita perché la libertà e la giustizia potessero essere scritte su questa Carta.”\nPiero Calamandrei\n\nUn abbraccio grande a questa nostra #Italia che ha bisogno di ciascuno di noi in questo momento difficile. #NoiDiAvis ci saremo sempre.\n#escosoloperdonare\n\n#Avis #donazione #sangue #plasma #piastrine #ig_avis #rossosangue #gialloplasma",1153,0.0
2,578,2020-03-14 00:00:00,avisnazionale,"🔝💪💛 #escosoloperdonare\nGrazie a tutti per l'incredibile risposta che state dando in tutto il paese. Siete fantastici. Se a volte trovate i numeri occupati, o la risposta non è immediata, considerate che ci state chiamando in tantissimi. Grazie per la pazienza e la generosità. #AVIS #coronavirus #donazione #sangue #plasma #piastrine #ig_avis",983,0.0
3,577,2020-03-19 00:00:00,avisnazionale,"#19marzo\nCaro papà,\nla mamma mi ha detto che non ci possiamo vedere in questo periodo perché tu in ospedale combatti un brutto mostro #coronavirus, che si appiccica alle braccia dei papà e delle mamme e poi si ammalano i bambini e i nonni. Allora aspetto, papà, e faccio la coraggiosa e non piango, anche se ho come un buco allo stomaco quando ti penso. Oggi è la tua festa, e ti ho fatto un bellissimo disegno. Con la mamma lo abbiamo appeso sopra il tuo letto per quando tornerai a casa. Un abbraccio grande quanto il mondo.\n\nA tutti i #medici e gli #infermieri che sono in prima linea notte e giorno per salvarci la vita.\nAiutiamoli: #restiamoacasa\n#escosoloperdonare #avis #donazione #sangue #plasma #piastrine #ig_avis",980,0.0
4,576,2020-03-12 00:00:00,avisnazionale,"#escosoloperdonare\nSiamo un po' tutti spaventati, ammettiamolo. Le città italiane non si vedevano così da quando d'agosto si svuotavano completamente e Adriano Celentano cantava ""Azzurro"".\nFa un po' impressione, è vero.\nMa sappiamo tutti che passerà, che dipende da tutti noi, e che noi di #Avis siamo doppiamente previdenti, per noi e per i nostri riceventi. Le donazioni proseguono regolarmente, su prenotazione, e riusciremo a garantire il sangue per tutti. Perché #NoisiamoAvis, e non ci ferma nessuno.\n#donazione #sangue #plasma #piastrine #ig_avis",917,0.0


,index,date,username,tweet,replies_count,sentiment
0,554,2020-04-20 00:00:00,avisnazionale,"⁉ #IlPresidenteRisponde #Avis\nRisposta:\nI #test #sierologici sono test diagnostici che si eseguono normalmente in clinica e servono per misurare soprattutto all’interno del siero e del plasma dei pazienti o dei soggetti sani le quantità di proteine presenti.\nQueste proteine sono di diversa natura (come l’albumina che è una proteina usata abitualmente dall’organismo per la sua sopravvivenza) oppure si sviluppano durante processi infiammatori (per esempio la proteina C reattiva) oppure sono proteine abbondantemente presenti nel siero, come gli anticorpi e le immunoglobuline, che sono risposte immunitarie del nostro organismo a insulti di tipo infettivo, che danno la cosiddetta “memoria immunologica” (che può essere recente o che può durare per tutta la vita) -\n-\n-\n#donazione #plasma #sangue #piastrine #ig_avis #escosoloperdonare #coronavirus",32,0.0
1,578,2020-03-14 00:00:00,avisnazionale,"🔝💪💛 #escosoloperdonare\nGrazie a tutti per l'incredibile risposta che state dando in tutto il paese. Siete fantastici. Se a volte trovate i numeri occupati, o la risposta non è immediata, considerate che ci state chiamando in tantissimi. Grazie per la pazienza e la generosità. #AVIS #coronavirus #donazione #sangue #plasma #piastrine #ig_avis",28,0.0
2,580,2020-03-15 00:00:00,avisnazionale,"#Avis ha 3400 sedi sul territorio nazionale, tutte gestite da volontari (con l'aiuto, in quelle più grandi, di qualche dipendente). Persone che hanno deciso di dedicare ad AVIS parte del loro tempo libero anche al tempo del #coronavirus. A tutti loro va la nostra immensa gratitudine. Senza di loro Avis non esisterebbe.😍 #NoiSiamoAvis #escosoloperdonare #donazione #sangue #plasma #piastrine #ig_avis",22,0.0
3,575,2020-03-10 00:00:00,avisnazionale,"Carissimi amici, ci siamo creati un Hashtag da donatori. Perché è vero che stiamo a casa e che dobbiamo tutti seguire alla lettera le regole, ma LA DONAZIONE DI SANGUE è un motivo per poter uscire. Le trasfusioni sono un Livello Essenziale di Assistenza, che deve essere garantito, e noi non ci fermeremo. #escosoloperdonare #AVIS #donazione #sangue #coronavirus",22,0.0
4,572,2020-03-14 00:00:00,avisnazionale,"📌🔴 #escosoloperdonare\nAnche dalle comunità di nuovi cittadini si risponde alle regole di contenimento del virus ed all'appello alla donazione.\nSono giorni difficili, ma stiamo vedendo ovunque gesti di grande generosità. Grazie davvero a tutti #Avis #Coronavirus #andratuttobene\n😍#donazione #sangue #plasma #piastrine #ig_avis",13,0.0


,index,date,username,tweet,likes_count,sentiment
0,492,2020-01-15 00:00:00,fidasdonatorisangue,Il Ministro della Salute Roberto Speranza ha invitato tutti i cittadini a prendere in considerazione la donazione del sangue come obiettivo personale per il 2020. \nLeggi il testo completo sul sito: fidas.it\n#fidas #giovaniFIDAS #donavita #donasangue #donazionesangue\n#donasanguedonavita #ministerodellasalute \n#Repost @ministerosalute ・・・\nDonare il sangue è importante tutto l'anno. Significa dare forma concreta alla solidarietà e offrire un aiuto prezioso a chi ne ha davvero bisogno. \nIn questo inizio 2020 decidi di fare un gesto che fa bene a te e fa bene agli altri: diventa donatore!\n\n@centronazionalesangue @crocerossaitaliana @avisnazionale @fratresnazionale @fidasdonatorisangue\n\n#MinisteroDellaSalute\n#SaluteSocial\n#donazionisangue #donatoridisangue #donaresangue\n#buonipropositi,131,0.0
1,476,2020-12-24 00:00:00,fidasdonatorisangue,"Auguri a tutti voi, cari donatori! 💓 Condividiamo con piacere la lettera di auguri del Presidente Nazionale FIDAS Giovanni Musso, a tutti i Consiglieri, i Presidenti e i Volontari di FIDAS. Nella biografia del nostro profilo, il link diretto per poter leggere la lettera. \n#FIDAS #GiovaniFIDAS #volontari #volontariato #DonaVita #donatoridisangue #oggidono #natale2020 #natale #santaclaus #santaiscoming #merryxmas #xmas",121,0.0
2,458,2020-02-25 00:00:00,fidasdonatorisangue,Pochi e semplici comportamenti da adottare per prevenire la diffusione del nuovo coronavirus e alcune semplici regole da ricordare in vista della donazione di sangue ed emocomponenti. #DonaVita\n.\n.\n.\n#coronavirus #nuovocoronavirus #influenza #FIDAS #donazionisangue #donasangue #donaplasma #donapiastrine #donatoridisangue #donasanguedonavita,109,0.0
3,354,2020-05-16 00:00:00,fidasdonatorisangue,"Grazie ad @our_dreams_15.01.2016\nperché ha voluto condividere con noi la sua prima aferesi avvenuta proprio in questi giorni. Sì, perché non solo si può continuare a donare nonostante la pandemia, ma per via della pandemia molti di noi hanno maggior tempo libero a propria disposizione e questa potrebbe essere una scusa in più per sperimentare nuove tipologie di donazioni non ancora provate prima. Ad esempio la donazione in aferesi del solo plasma, o delle sole piastrine. \nI @giovanifidas, grazie ad @our_dreams_15.01.2016, ci dimostrano che è possibile! 💪😊\n#DonaVita #GiovaniFIDAS\n#aferesi #donareplasma\n#donazione #donazioneplasma #FIDASPartenopea #igersnapoli #ignapoli .\n.\n.\n#Repost \nLa mia prima aferesi ❤️ ""Le emozioni più belle sono quelle che non puoi spiegare."" ~Charles Baudelaire.\n#IORESTOACASA #escosoloperdonare #oggicosì #oggidono #sangue #blood #covid_19 #distantimauniti #fidas #picoftheday📷 #instapic #instalover #dono\n#campania #napoli #ponticelli",75,0.0
4,325,2021-03-02 00:00:00,fidasdonatorisangue,"""Beh, che dire? 🤔 Mi chiamo Anna e per un po' di mesi posso ancora considerarmi una ""under 28"" 🤪 ...Se sono una volontaria ed anche una donatrice di sangue lo devo al mio mitico papino 🥰, che ora mi guarda da lassù cercando di indirizzarmi per le vie tortuose della vita... Fiera ed onorata di essere donatrice da sempre perché con un piccolo gesto si possono compiere grandi passi!!✌🏻😎"".\nAnna Salatin, donatrice e volontaria @fidastreviso \n.\n.\n.\n#FIDAS #FIDASTreviso #GiovaniFIDAS #volontaria #volontariato #donaresangue #oggidono #treviso #igerstreviso #igtreviso #veneto #igersveneto #igveneto",71,0.0


,index,date,username,tweet,replies_count,sentiment
0,476,2020-12-24 00:00:00,fidasdonatorisangue,"Auguri a tutti voi, cari donatori! 💓 Condividiamo con piacere la lettera di auguri del Presidente Nazionale FIDAS Giovanni Musso, a tutti i Consiglieri, i Presidenti e i Volontari di FIDAS. Nella biografia del nostro profilo, il link diretto per poter leggere la lettera. \n#FIDAS #GiovaniFIDAS #volontari #volontariato #DonaVita #donatoridisangue #oggidono #natale2020 #natale #santaclaus #santaiscoming #merryxmas #xmas",4,0.0
1,492,2020-01-15 00:00:00,fidasdonatorisangue,Il Ministro della Salute Roberto Speranza ha invitato tutti i cittadini a prendere in considerazione la donazione del sangue come obiettivo personale per il 2020. \nLeggi il testo completo sul sito: fidas.it\n#fidas #giovaniFIDAS #donavita #donasangue #donazionesangue\n#donasanguedonavita #ministerodellasalute \n#Repost @ministerosalute ・・・\nDonare il sangue è importante tutto l'anno. Significa dare forma concreta alla solidarietà e offrire un aiuto prezioso a chi ne ha davvero bisogno. \nIn questo inizio 2020 decidi di fare un gesto che fa bene a te e fa bene agli altri: diventa donatore!\n\n@centronazionalesangue @crocerossaitaliana @avisnazionale @fratresnazionale @fidasdonatorisangue\n\n#MinisteroDellaSalute\n#SaluteSocial\n#donazionisangue #donatoridisangue #donaresangue\n#buonipropositi,2,0.0
2,325,2021-03-02 00:00:00,fidasdonatorisangue,"""Beh, che dire? 🤔 Mi chiamo Anna e per un po' di mesi posso ancora considerarmi una ""under 28"" 🤪 ...Se sono una volontaria ed anche una donatrice di sangue lo devo al mio mitico papino 🥰, che ora mi guarda da lassù cercando di indirizzarmi per le vie tortuose della vita... Fiera ed onorata di essere donatrice da sempre perché con un piccolo gesto si possono compiere grandi passi!!✌🏻😎"".\nAnna Salatin, donatrice e volontaria @fidastreviso \n.\n.\n.\n#FIDAS #FIDASTreviso #GiovaniFIDAS #volontaria #volontariato #donaresangue #oggidono #treviso #igerstreviso #igtreviso #veneto #igersveneto #igveneto",1,0.0
3,458,2020-02-25 00:00:00,fidasdonatorisangue,Pochi e semplici comportamenti da adottare per prevenire la diffusione del nuovo coronavirus e alcune semplici regole da ricordare in vista della donazione di sangue ed emocomponenti. #DonaVita\n.\n.\n.\n#coronavirus #nuovocoronavirus #influenza #FIDAS #donazionisangue #donasangue #donaplasma #donapiastrine #donatoridisangue #donasanguedonavita,1,0.0
4,45,2020-12-06 00:00:00,fidasdonatorisangue,Grazie per il tuo dono! Fondamentale per i pazienti che necessitano di terapie trasfusionali! 😊🙌,0,0.0


In [25]:
#Top 5 posts - Twitter
orgs_tw = list(inst_tw.username.unique())
metric_tw = ['likes_count','replies_count','retweets_count']
top5_posts_tw = []
for org in orgs_tw:
    for metric in metric_tw:
        top5_posts_tw.append(top5_posts(inst_tw,org,metric))

In [26]:
for i in top5_posts_tw:
    display(i)

,index,date,username,tweet,likes_count,sentiment
0,2262,2018-06-26,avis_milano,"Donatori di sangue, in Italia è ancora crisi: 8 mila in meno nel 2017 https://t.co/7ZiaQdFuzO #donazione #sangue #donatori",4,0.0
1,2128,2014-06-05,avis_milano,Sabato 14 giugno alle 11.30 in Stazione Centrale #flashmob di #AvisMIlano per #giornatadonatore. Dress code maglietta rossa. Vi aspettiamo!!,3,0.0
2,1289,2015-01-26,avis_milano,la conferenza stampa di #avisMilano al @TGR_rai della #Lombardia http://t.co/Ies1yKZQjA,1,0.0
3,709,2016-01-06,avis_milano,Anche negli Usa cade il divieto di donazione per gli omosessuali https://t.co/hXeXi1PG03 #blooddonor #diritti #lgbt,1,0.0
4,1281,2014-09-09,avis_milano,Presidente #avismilano Massimo Molla: lo sport è ottimo veicolo per promuovere donazione di sangue #innovationrunning,1,0.0


,index,date,username,tweet,replies_count,sentiment
0,568,2014-03-06,avis_milano,La relazione del presidente di #AvisMilano Luca Munari all'assemblea annuale dell'associazione http://t.co/tZHypXFkgI,1,0.0
1,2128,2014-06-05,avis_milano,Sabato 14 giugno alle 11.30 in Stazione Centrale #flashmob di #AvisMIlano per #giornatadonatore. Dress code maglietta rossa. Vi aspettiamo!!,1,0.0
2,1291,2015-03-20,avis_milano,Grazie a @uniiulm e @iulmstudenti per il bellissimo percorso fatto insieme sulle campagne di comunicazione per la donazione! #avisMilano,1,0.0
3,579,2014-01-27,avis_milano,#Buongiorno e #Buonasettimana è online il blog di #Avismilano con novità storie e informazioni. Correte a leggerlo! http://t.co/HV8JndyjkB,1,0.0
4,569,2014-03-05,avis_milano,Dall'8 al 10 marzo si tengono le elezioni per il rinnovo del consiglio di #AvisMilano. Votare è importante! http://t.co/hDspa9WxH4,1,0.0


,index,date,username,tweet,retweets_count,sentiment
0,1285,2013-10-30,avis_milano,Voto definitivo al #senato. Le donazioni hanno valore per la pensione #avismilano http://t.co/I6bxQHk9Jq,8,0.0
1,2128,2014-06-05,avis_milano,Sabato 14 giugno alle 11.30 in Stazione Centrale #flashmob di #AvisMIlano per #giornatadonatore. Dress code maglietta rossa. Vi aspettiamo!!,5,0.0
2,709,2016-01-06,avis_milano,Anche negli Usa cade il divieto di donazione per gli omosessuali https://t.co/hXeXi1PG03 #blooddonor #diritti #lgbt,3,0.0
3,319,2016-07-12,avis_milano,Emergenza sangue in #Puglia per lo scontro dei treni tra Andria e Corato https://t.co/VBchtdpmrX #emergenzasangue,3,0.0
4,2262,2018-06-26,avis_milano,"Donatori di sangue, in Italia è ancora crisi: 8 mila in meno nel 2017 https://t.co/7ZiaQdFuzO #donazione #sangue #donatori",2,0.0


,index,date,username,tweet,likes_count,sentiment
0,4414,2020-02-28,fidasnazionale,"Questa notte ci ha lasciati il nostro amato Presidente, Aldo Ozino Caligaris. Un abbraccio da parte di tutta la ""grande famiglia FIDAS"", come amava chiamarla lui, giunga a sua moglie e ai suoi figli. #FIDAS #AldoOzinoCaligaris https://t.co/iV8RJcH4cD",32,0.0
1,4323,2017-02-17,fidasnazionale,Anche i nostri amici a quattro zampe possono diventare donatori di sangue #festadelgatto #Venerdi17 https://t.co/DRYz9CXYtO,24,0.0
2,4231,2020-08-28,fidasnazionale,La donazione di sangue e plasma è indispensabile in occasione di trapianto d'organo. Complimenti all'equipe che ha permesso questo eccellente risultato per 5 bambini e ragazzi... e grazie a tutti i donatori! #DonaVita,20,0.0
3,4175,2020-10-02,fidasnazionale,"Auguri a tutti i nonni! Voi che siete un sostegno per la famiglia, che accompagnate figli e nipoti… grazie per il dono che siete! 💞 #2ottobre #Festadeinonni #FIDAS https://t.co/1iM4clzbC6",19,0.0
4,4048,2019-11-03,fidasnazionale,"Non è retorica: siamo veramente convinti che siate voi, i volontari, il vero DNA della FIDAS. In voi risiede il segreto dello #stileFIDAS. Grazie a tutti voi per questo splendido primo appuntamento del #FIDASLab https://t.co/Gh0viEAIKC",16,0.0


,index,date,username,tweet,replies_count,sentiment
0,4414,2020-02-28,fidasnazionale,"Questa notte ci ha lasciati il nostro amato Presidente, Aldo Ozino Caligaris. Un abbraccio da parte di tutta la ""grande famiglia FIDAS"", come amava chiamarla lui, giunga a sua moglie e ai suoi figli. #FIDAS #AldoOzinoCaligaris https://t.co/iV8RJcH4cD",3,0.0
1,2870,2015-06-02,fidasnazionale,Iscriviti sul sito di @radiodeejay e il 12 giugno dona il sangue con FIDAS e il @triomedusa http://t.co/MnjScE63KF,3,0.0
2,1269,2020-04-23,fidasnazionale,"@GiacominoM @Laura6976 @avisnazionale Di nulla, grazie a te per il tuo costante dono! 🙏",2,0.0
3,1262,2021-03-10,fidasnazionale,@lorenzcolman 1/2 E' importante essere sinceri rispetto al proprio stato di salute e ai comportamenti adottati perché le risposte sono utili alla tutela della salute del donatore ma anche del ricevente. Chi ha fatto piercing può tornare a donare in completa sicurezza dopo 4 mesi.,2,0.0
4,1260,2020-04-23,fidasnazionale,"@GiacominoM @Laura6976 @avisnazionale @ADVSangue Siamo certi che suo padre sarà orgoglioso di lei, anzi, che già lo sia per quanto ha potuto donare in questi anni! 😊",2,0.0


...

,index,date,username,tweet,replies_count,sentiment
0,2118,2015-06-02,nybloodcenter,Save lives on June 17th at @NSLIJCC. Walk in blood donors are welcome! https://t.co/oAqWxG0kiX,10,1.00000
1,4451,2020-11-26,nybloodcenter,"Do your part NYC, blood donors save lives! We’re excited to partner with our friends, @NYCMayor, @NYCMayorsOffice, and @nycgov! Learn about the #GiveBloodNYC sweepstakes here: https://t.co/8SuYooVAKv",7,0.46875
2,4450,2020-11-26,nybloodcenter,"Do your part NYC, blood donors save lives! We’re excited to partner with our friends, @NYCMayor, @NYCMayorsOffice, and @nycgov! Learn about the #GiveBloodNYC sweepstakes here: https://t.co/8SuYooVAKv",7,0.46875
3,2105,2016-04-21,nybloodcenter,Save lives TODAY St. John's University until 4:00! Walk in blood donors are welcome on the Blood Vessel bus in... https://t.co/GBwAf4GQmw,6,0.80000
4,4462,2020-03-30,nybloodcenter,"Nicole knows just how important donating blood is from being a donor, a New York Blood Center team member, and recently, a recipient. Make sure that new moms like Nicole have blood available to them as needed during some childbirth. Make your appointment: https://t.co/oXznn6GXbQ https://t.co/P9K5JCY6mh",5,0.21039


,index,date,username,tweet,retweets_count,sentiment
0,4344,2017-05-08,nybloodcenter,Save lives with the @NYPD42pct 12-6 Today on the busmobile outside the precinct. Walk in blood donors are welcome and thanks to the #nypd https://t.co/sgLJeuakZD,23,0.333333
1,4462,2020-03-30,nybloodcenter,"Nicole knows just how important donating blood is from being a donor, a New York Blood Center team member, and recently, a recipient. Make sure that new moms like Nicole have blood available to them as needed during some childbirth. Make your appointment: https://t.co/oXznn6GXbQ https://t.co/P9K5JCY6mh",21,0.210390
2,4461,2020-03-30,nybloodcenter,"Nicole knows just how important donating blood is from being a donor, a New York Blood Center team member, and recently, a recipient. Make sure that new moms like Nicole have blood available to them as needed during some childbirth. Make your appointment: https://t.co/oXznn6GXbQ https://t.co/P9K5JCY6mh",21,0.210390
3,4451,2020-11-26,nybloodcenter,"Do your part NYC, blood donors save lives! We’re excited to partner with our friends, @NYCMayor, @NYCMayorsOffice, and @nycgov! Learn about the #GiveBloodNYC sweepstakes here: https://t.co/8SuYooVAKv",17,0.468750
4,4450,2020-11-26,nybloodcenter,"Do your part NYC, blood donors save lives! We’re excited to partner with our friends, @NYCMayor, @NYCMayorsOffice, and @nycgov! Learn about the #GiveBloodNYC sweepstakes here: https://t.co/8SuYooVAKv",17,0.468750


,index,date,username,tweet,likes_count,sentiment
0,4028,2019-07-25,redcrosseny,"In honor of #NationalIceCreamMonth we’re thrilled to report that our #GiveAPintGetAPint partnership with @StewartsShops resulted in over 5,000 units of blood collected during the month of June!! And over 20% of those units came from first time blood donors!! 😮🍦🙏 https://t.co/OlsNlXuEui",16,0.664062
1,2390,2020-03-15,redcrosseny,"If you are an #Eligible #Blood #Donor candidate, search for a location near you to get out there and donate! Help your community today. Busting Blood Donation Myths: https://t.co/87f9fSC37B via @YouTube",4,0.125000
2,2389,2020-03-18,redcrosseny,"Blood donors and blood drive hosts play a critical role in maintaining a sufficient blood supply. In the face of a severe blood shortage, we need our partners now more than ever to keep hosting blood drives for patients who rely on lifesaving blood. Thank you @SSCityCenter!!",4,0.390625
3,2063,2019-10-09,redcrosseny,"The Kirkland Blood Committee has organized hundreds of local blood drives over the last 50 years &amp; helped collect approx. 45,000 units of blood! 1 blood donation can help 3 patients, so just think about how many lives these volunteers have touched! 😮 https://t.co/oYYawb3P5d https://t.co/VkwjfR2D4O",3,0.208333
4,2054,2021-02-17,redcrosseny,We’re seeking #volunteers in the #CapitalRegion to support blood drives and the Blood Donation Center in #Albany. Volunteers take temperatures of donors upon entry and help with registration. Sound like you? Visit https://t.co/C8GHEVgHwZ or contact Lauren.Whitman@redcross.org https://t.co/nBoaKvc1FF,3,0.150000


,index,date,username,tweet,replies_count,sentiment
0,2390,2020-03-15,redcrosseny,"If you are an #Eligible #Blood #Donor candidate, search for a location near you to get out there and donate! Help your community today. Busting Blood Donation Myths: https://t.co/87f9fSC37B via @YouTube",2,0.125000
1,4028,2019-07-25,redcrosseny,"In honor of #NationalIceCreamMonth we’re thrilled to report that our #GiveAPintGetAPint partnership with @StewartsShops resulted in over 5,000 units of blood collected during the month of June!! And over 20% of those units came from first time blood donors!! 😮🍦🙏 https://t.co/OlsNlXuEui",1,0.664062
2,2054,2021-02-17,redcrosseny,We’re seeking #volunteers in the #CapitalRegion to support blood drives and the Blood Donation Center in #Albany. Volunteers take temperatures of donors upon entry and help with registration. Sound like you? Visit https://t.co/C8GHEVgHwZ or contact Lauren.Whitman@redcross.org https://t.co/nBoaKvc1FF,1,0.150000
3,861,2021-01-29,redcrosseny,We were joined by an amazing group of volunteers from Six Flags Great Escape as we wrapped up our series of #DistanceSavesLives blood drives. Thank you to all of our community partners and blood donors for helping us collect lifesaving blood for patients in need. https://t.co/hlN2bzLv3d,0,0.700000
4,2389,2020-03-18,redcrosseny,"Blood donors and blood drive hosts play a critical role in maintaining a sufficient blood supply. In the face of a severe blood shortage, we need our partners now more than ever to keep hosting blood drives for patients who rely on lifesaving blood. Thank you @SSCityCenter!!",0,0.390625


,index,date,username,tweet,retweets_count,sentiment
0,2054,2021-02-17,redcrosseny,We’re seeking #volunteers in the #CapitalRegion to support blood drives and the Blood Donation Center in #Albany. Volunteers take temperatures of donors upon entry and help with registration. Sound like you? Visit https://t.co/C8GHEVgHwZ or contact Lauren.Whitman@redcross.org https://t.co/nBoaKvc1FF,3,0.150000
1,4028,2019-07-25,redcrosseny,"In honor of #NationalIceCreamMonth we’re thrilled to report that our #GiveAPintGetAPint partnership with @StewartsShops resulted in over 5,000 units of blood collected during the month of June!! And over 20% of those units came from first time blood donors!! 😮🍦🙏 https://t.co/OlsNlXuEui",2,0.664062
2,2390,2020-03-15,redcrosseny,"If you are an #Eligible #Blood #Donor candidate, search for a location near you to get out there and donate! Help your community today. Busting Blood Donation Myths: https://t.co/87f9fSC37B via @YouTube",2,0.125000
3,2389,2020-03-18,redcrosseny,"Blood donors and blood drive hosts play a critical role in maintaining a sufficient blood supply. In the face of a severe blood shortage, we need our partners now more than ever to keep hosting blood drives for patients who rely on lifesaving blood. Thank you @SSCityCenter!!",2,0.390625
4,1124,2019-07-12,redcrosseny,Have you heard? We're facing a #BloodEmergency! Don’t wait – schedule a blood donation today at https://t.co/6UDN3mtIwx or call 1-800-RED CROSS (1-800-733-2767).#BattleoftheBadges https://t.co/WdwbsninLM,2,0.000000


In [27]:
#top 5 mentions - Instagram
orgs_ig = list(mentions_ig.cited.unique())
metric_ig = ['likes_count','replies_count']
top5_mentions_ig = []
for org in orgs_ig:
    for metric in metric_ig:
        top5_mentions_ig.append(top5_mentions(mentions_ig,org,metric))

In [28]:
for i in top5_mentions_ig:
    display(i)

,index,date,username,cited,tweet,likes_count,sentiment
0,2296,2020-03-18 00:00:00,michelangelo_tommaso,avisnazionale,"Ciao a tutti!!! La smettete di andare in giro e fare i coglioni irresponsabili!?!? Se proprio volete uscire seguite le linee guida dell’Avis, seguite l’hashtag #escosoloperdonare! C’è bisogno di #sangue in questo momento, e c’è bisogno di aiutare gli ospedali! Andate sul sito di @avisnazionale e troverete tutte le info per donare! Anche gli ospedali hanno bisogno di aiuto!!! I medici stanno lavorando in condizioni di estrema difficoltà e c’è carenza di mascherine, camici e schermi protettivi. Aiutiamo chi è in prima linea!!! Sui siti delle regioni ci sono tutte le informazioni per effettuare le donazioni. Più facciamo adesso e più ne usciremo prima!! Forzaaaaaa 💪🏻💪🏻💪🏻💪🏻\n#escosoloperdonare #avis #policlinicogemelli #spallanzani #cotugnonapoli #ospedalesaccomilano #doniamo #facciamolanostraparte",2766,0.000000
1,185,2020-12-28 00:00:00,marianna_aprile,avisnazionale,"L’alba, stamattina, era bianca e da romanzo russo. E l’ho camminata fino a Lambrate per donare il sangue (no, “lui” è andato dopo, prima che me lo chiediate). Come sempre, le cose che fai per gli altri diventano doni che fai a te, perché questa passeggiata è un regalo di fine anno bellissimo. \nPs: santo subito il dottore per la pazienza che ha avuto con la mia paura per l’ago. 🙏\n@avisnazionale",1509,0.416667
2,458,2020-03-13 00:00:00,claudiolauretta,avisnazionale,Per chi può e per chi si sente di donare! AVIS @avisnazionale \nC’è bisogno di donazioni! ❣️#avis #donazionisangue #donatorisangue #sangueeplasma #avistortona #iorestoacasa #tuttoandràbene 💪🏻🇮🇹\nBUONA GIORNATA A TUTTI!!! ☀️,826,0.000000
3,187,2020-12-16 00:00:00,luisabertoldo,avisnazionale,".\nQuesta foto non vincerà il primo premio in bellezza ma spero in efficacia perché vuole ricordarci di donare il sangue🩸 \nPer essere idonei servono alcune precise caratteristiche che condivido qui perché a me più volte ne sono mancate alcune:\n-non essere anemici\nchissà quanti di noi non sanno di esserlo almeno un pó. Per evitare questa carenza che non fa mai bene un po’ di carne rossa (poca ma buona) o in alternativa pesci, legumi e verdure che aiutano, o supporti chimici come ultima chance.\n-non aver partorito da poco, o essere incinta\n-essere in un preciso momento lontano dal ciclo\n-essere lontani da viaggi in paesi tropicali\n-pesare più di 50 kg\n-non avere specifiche malattie/assunzioni di farmaci\n-tendenzialmente essere dai 18 ai 60 anni\n-avere il desiderio di fare un gesto che possa aiutare gli altri andando oltre la pigrizia o la sensazione poco piacevole di farsi infilare un piccolo ago sul braccio (per i più paurosi pensate che a chi serve quel sangue ha dovuto far molto più di questo).\nGuardate questo elenco e se capite che voi potreste essere idonei andate a donare, altre persone vorrebbero ma non possono farlo e prenderebbero volentieri il vostro posto.\nSul sito @avisnazionale trovate tutte le informazioni.\nAh dimenticavo! dicono che donare il sangue faccia anche bene alla salute oltre che al cuore✌🏻",790,0.262500
4,1770,2020-03-15 00:00:00,lorettamira,avisnazionale,"Questa mattina sono uscita di casa e sono andata a donare. Sinceramente ero terrorizzata e molto preoccupata. Poi, dopo il colloquio con il medico e gli infermieri, ho capito che non devo avere paura di donare e che non ci sono pericoli. Quindi, se potete, donate, ora più che mai tantissime persone hanno bisogno di sangue💗💗💗buona domenica a tutti amici cari, ANDRÀ TUTTO BENE 💗💗💗\n#avis #donazione #donazionedisangue #andratuttobene #iorestoacasa #felicità #amicizia #sangue #domenica #torinoèlamiacittà #plasma #donazionediplasma #nole #aiutareglialtri #aiuto #amorevero #io #semplicementeio #andràtuttobene #lemiegiornate #marzo #aspettandolaprimavera #famiglia❤️ #incasa #andiamoadonareilsangue #fatelo #avisnazionale #happy #blood #escosoloperdonare",721,0.800000


,index,date,username,cited,tweet,replies_count,sentiment
0,2296,2020-03-18 00:00:00,michelangelo_tommaso,avisnazionale,"Ciao a tutti!!! La smettete di andare in giro e fare i coglioni irresponsabili!?!? Se proprio volete uscire seguite le linee guida dell’Avis, seguite l’hashtag #escosoloperdonare! C’è bisogno di #sangue in questo momento, e c’è bisogno di aiutare gli ospedali! Andate sul sito di @avisnazionale e troverete tutte le info per donare! Anche gli ospedali hanno bisogno di aiuto!!! I medici stanno lavorando in condizioni di estrema difficoltà e c’è carenza di mascherine, camici e schermi protettivi. Aiutiamo chi è in prima linea!!! Sui siti delle regioni ci sono tutte le informazioni per effettuare le donazioni. Più facciamo adesso e più ne usciremo prima!! Forzaaaaaa 💪🏻💪🏻💪🏻💪🏻\n#escosoloperdonare #avis #policlinicogemelli #spallanzani #cotugnonapoli #ospedalesaccomilano #doniamo #facciamolanostraparte",137,0.0
1,458,2020-03-13 00:00:00,claudiolauretta,avisnazionale,Per chi può e per chi si sente di donare! AVIS @avisnazionale \nC’è bisogno di donazioni! ❣️#avis #donazionisangue #donatorisangue #sangueeplasma #avistortona #iorestoacasa #tuttoandràbene 💪🏻🇮🇹\nBUONA GIORNATA A TUTTI!!! ☀️,84,0.0
2,1995,2020-03-30 00:00:00,un_altro_profilo_inutile,avisnazionale,".\nSe vi siete annoiati di stare a casa, una buona scusa per uscire ve la suggerisco io: andate a donare sangue. 😉\n#PASSAPAROLA\n.\n#escosoloperdonare #DiversamenteChiaraFerragni #poinondire #avisbrindisi #avis #avisnazionale #donaora #donaadesso #donaspesso #dona #sangue #donatori #donazionesangue #ospedale",80,0.0
3,437,2021-02-17 00:00:00,claudiaconte.it,avisnazionale,“Dona il sangue. Salva una Vita”\nGrazie al mio amico @lucacapuanofficial per avermi consigliato @ematosfidas 🩸 consiglio a tutti!!!\n\n.\n.\n.\n#blood #sangue #donazionesangue #avis #donazione #volontariato #donatoridisangue #plasma #iodono #solidarietà #donare #donasangue #donaresangue #failadifferenza #donarefabene #giveblood #donaora #donaspesso #donatori #blood #dona #donazionedelsangue #amore #donazioneplasma #blooddonation #avisnazionale #salvaunavita #fatebenefratelli #ospedale #ematologia,78,0.0
4,1641,2021-02-17 00:00:00,claudiaconte.it,avisnazionale,“Dona il sangue. Salva una Vita”\nGrazie al mio amico @lucacapuanofficial per avermi consigliato @ematosfidas 🩸 consiglio a tutti!!!\n\n.\n.\n.\n#blood #sangue #donazionesangue #avis #donazione #volontariato #donatoridisangue #plasma #iodono #solidarietà #donare #donasangue #donaresangue #failadifferenza #donarefabene #giveblood #donaora #donaspesso #donatori #blood #dona #donazionedelsangue #amore #donazioneplasma #blooddonation #avisnazionale #salvaunavita #fatebenefratelli #ospedale #ematologia,78,0.0


,index,date,username,cited,tweet,likes_count,sentiment
0,2242,2020-03-19 00:00:00,stefanobruce,fidasdonatorisangue,Prendetevi 5 minuti chiamate il +39 339 3607451 e prenotate la vostra donazione ! C’è bisogno di sangue! #escosoloperdonare #escosoloperdonaresangue #avantifidasterme #donatoridisangue #fidasterme #fidasdonatorisangue #fidaszonaest,121,0.00
1,2191,2020-03-18 00:00:00,vittorio_hamarz,fidasdonatorisangue,Oggi si dona per un futuro migliore! @ufficiostampa_fatebenefratelli @ematosfidas @vittorio_hamarz @fidasdonatorisangue @nazionaleattori @interiosline @primalandsafaris @shahpartherealtor @shahparostovar @muscialexa @teatro.it @noidiroma @wantedinrome #actor #actorslife #treattorinaffitto #3attorinaffitto #donailsangue #donalavita #donarefabene #likeforlikes #like4tags #likebacknow #escosoloperdonare #hamarzvasfi #vittoriohamarzvasfi,104,0.00
2,447,2021-01-23 00:00:00,giovanifidas,fidasdonatorisangue,"“Sono classe 1993, studentessa universitaria di geologia. Faccio volontariato nella mia federata di Bologna da ormai 6 anni, e sono donatrice da quando ne ho avuto la possibilità”.\nTeresa Oreade Grillo, volontaria e donatrice @fidas_bologna \n.\n.\n.\n#FIDAS #fidasbologna #bologna #igersbologna #igbologna #donaresangue #testimonianza #GiovaniFIDAS #volontaria #volontariato #geologia\nReposted from @fidasdonatorisangue",65,0.00
3,2018,2020-03-21 00:00:00,lu.mol13,fidasdonatorisangue,distanti ma uniti.\n\n#bome2020 #giovanifidas #iorestoacasa #escosoloperdonare @giovanifidas @fidasdonatorisangue,63,0.00
4,1702,2020-12-21 00:00:00,_robiro___,fidasdonatorisangue,"#Repost @fidasdonatorisangue \nGRAZIE FIDAS ❤️❤️\n・・・\n""Ho solo 26 anni, ma se mi chiedi il numero di donazioni effettuate non so rispondere: sono talmente tante che ho già perso il conto! Nella vita mi occupo di Business Analytics, mentre nel tempo libero mi alterno tra volontariato e calcetto. Sono la coordinatrice Giovani FIDAS di Bologna e da ottobre vivo una nuova avventura, avendo ricevuto l'incarico di segretaria del Coordinamento Nazionale Giovani FIDAS"".\nRoberta Antonia Maida, volontaria @fidas_bologna e segretaria @giovanifidas\nE tu, a quante donazioni sei arrivat*? Ricordi il numero?\n.\n.\n.\n#bologna #igersbologna #igbologna #donaresangue #donavita #donatoridisangue #business #businessanalytics #calcetto #volontariato #GiovaniFidas #young #red",61,0.05


,index,date,username,cited,tweet,replies_count,sentiment
0,2191,2020-03-18 00:00:00,vittorio_hamarz,fidasdonatorisangue,Oggi si dona per un futuro migliore! @ufficiostampa_fatebenefratelli @ematosfidas @vittorio_hamarz @fidasdonatorisangue @nazionaleattori @interiosline @primalandsafaris @shahpartherealtor @shahparostovar @muscialexa @teatro.it @noidiroma @wantedinrome #actor #actorslife #treattorinaffitto #3attorinaffitto #donailsangue #donalavita #donarefabene #likeforlikes #like4tags #likebacknow #escosoloperdonare #hamarzvasfi #vittoriohamarzvasfi,21,0.00
1,1695,2020-12-08 00:00:00,fidaspartenopea,fidasdonatorisangue,Rendi il tuo natale più magico con FIDAS💪 seguici anche su TikTok: @fidas.partenopea\n#donatoridisangue #donor #fidaspartenopea #tiktok #fidasinthefuture\n\nE voi avete fatto l'albero??\nFateci vedere le vostre decorazioni!!! 💞\nTaggateci nei vostri post o stories natalizie... 🎄\nLe più belle avranno in palio la nostra pallina #ufficiale\n\n#christmas #christmasdecorations #christmasinfidas\n\n@fidasdonatorisangue @giovanifidas,7,0.00
2,1702,2020-12-21 00:00:00,_robiro___,fidasdonatorisangue,"#Repost @fidasdonatorisangue \nGRAZIE FIDAS ❤️❤️\n・・・\n""Ho solo 26 anni, ma se mi chiedi il numero di donazioni effettuate non so rispondere: sono talmente tante che ho già perso il conto! Nella vita mi occupo di Business Analytics, mentre nel tempo libero mi alterno tra volontariato e calcetto. Sono la coordinatrice Giovani FIDAS di Bologna e da ottobre vivo una nuova avventura, avendo ricevuto l'incarico di segretaria del Coordinamento Nazionale Giovani FIDAS"".\nRoberta Antonia Maida, volontaria @fidas_bologna e segretaria @giovanifidas\nE tu, a quante donazioni sei arrivat*? Ricordi il numero?\n.\n.\n.\n#bologna #igersbologna #igbologna #donaresangue #donavita #donatoridisangue #business #businessanalytics #calcetto #volontariato #GiovaniFidas #young #red",6,0.05
3,1705,2020-12-21 00:00:00,giovanifidas,fidasdonatorisangue,"""Ho solo 26 anni, ma se mi chiedi il numero di donazioni effettuate non so rispondere: sono talmente tante che ho già perso il conto! Nella vita mi occupo di Business Analytics, mentre nel tempo libero mi alterno tra volontariato e calcetto. Sono la coordinatrice Giovani FIDAS di Bologna e da ottobre vivo una nuova avventura, avendo ricevuto l'incarico di segretaria del Coordinamento Nazionale Giovani FIDAS"".\nRoberta Antonia Maida, volontaria @fidas_bologna e segretaria @giovanifidas\nE tu, a quante donazioni sei arrivat*? Ricordi il numero?\n.\n.\n.\n#bologna #igersbologna #igbologna #donaresangue #donavita #donatoridisangue #business #businessanalytics #calcetto #volontariato #GiovaniFidas #young #red\nReposted from @fidasdonatorisangue",6,0.05
4,1894,2020-06-02 00:00:00,fidaspartenopea,fidasdonatorisangue,Eccovi l'esperienza della nostra Ines pubblicata grazie a @donatorih24 🤩\n\nGrazie a @fidasdonatorisangue e @giovanifidas per tutto quello che fate. \nDONARE SANGUE È VITA!\n\n#oggidono #fidaspartenopea #fidas #escosoloperdonare #aferesis #donate #blooddonor,3,0.00


,index,date,username,cited,tweet,likes_count,sentiment
0,90,2020-12-01 00:00:00,fdny,newyorkbloodcenter,"New York City has an alarming shortage of donated blood due to the COVID-19 pandemic. The urgent need for donors continues. Every time you donate, you save a life. Give blood to help your fellow New Yorkers. For more information on how you can donate, visit NYBC.org/GiveBloodNYC and follow @newyorkbloodcenter. (Photo taken in 2017) #GiveBloodNYC",7802,0.109545
1,1380,2019-06-27 00:00:00,dr_mudgil,newyorkbloodcenter,"NEW YouTube/IGTV/Facebook joint is LIVE! ⁣\n⁣\nDonating blood is the most precious gift you can give a fellow human! In this video I discuss why it’s so important to donate, if you’re able.⁣\n⁣\n\n#blooddonor #blooddonation #americanredcross #newyorkbloodcenter #donateblood",1800,0.284470
2,1611,2019-07-07 00:00:00,dr_mudgil,newyorkbloodcenter,"All new episode of “The Dr. Mudgil Show” drops at 11 AM EST today on Zee TV! ⁣⁣\n⁣⁣\nIt’ll also be available on my YouTube channel, IGTV, and Facebook! 👊🏽⁣⁣\n⁣⁣\n⁣⁣\n⁣⁣#blooddonation #blooddonor #newyorkbloodcenter #redcross #thedrmudgilshow",1394,0.335227
3,1242,2019-12-13 00:00:00,soulexcelsior,newyorkbloodcenter,"Your time is precious, especially during the holiday season. Every second someone needs blood. Consider giving back the ultimate gift of love- the gift of life. Donate blood today. Blessings💜#blooddonation#blood#donateblood#blooddonor#giveblood#bloodbank#volunteer#donate#newyorkbloodcenter#blooddrive#givebloodsavelives#happyholidays#givebloodforchristmas#lifecoach#healersofinstagram#hypnotherapist#reikimaster#soulexcelsior.com",1240,0.166667
4,1513,2019-07-21 00:00:00,dr_mudgil,newyorkbloodcenter,"If you’re able, there’s no better gift you can give than a pint of blood! One pint of blood can save three lives! ⁣\n⁣\nIn the newest episode of the “The Dr. Mudgil Show” I discuss blood donation in detail and take you along with me to donate a pint of blood! Available on YouTube, IGTV, and Facebook!⁣\n⁣\n#blooddonor #blooddonation #newyorkbloodcenter #redcross #thedrmudgilshow",1165,0.137240


,index,date,username,cited,tweet,replies_count,sentiment
0,1005,2020-04-28 00:00:00,travelingwithjessica,newyorkbloodcenter,"For over 2 weeks, I went through what felt like the worse sickness of my life = COVID-19. I was congested to the point where I couldn’t breathe properly. I completely lost my sense of taste and smell, and could barely make it out of bed due to debilitating body aches. 😩 ·\n·\nFinally on my 12th day, I started to feel better. And with every passing day, my health was improving. I’d gone back to my doctor to get retested after 14 days and my results came back negative the second time—honestly, I cried as soon as I found out. Testing was extremely difficult—I was denied more than 2x before showing worse and worse symptoms. Fast forward, I couldn’t believe that I beat this crappy illness. Unfortunately, little did I know that I would lose both of my grandparents, just two days apart, to COVID complications. They weren’t staying with me, so my family and I will never truly know how they caught it. I was devastated, shocked, and in disbelief—I still am. 😢 As much as I wanted to desperately donate my plasma to at least one of my grandparents, time simply wasn’t on our side. 🥺\n·\nI can’t reverse time, but I’m grateful and happy to have made the decision to help others in need that have it worse than what I’ve gone through. 🙏🏼 ·\n·\nAt the end of the day, I hope you guys can look at this photo and feel compelled to donate plasma (only if you’re able to), and especially if you’ve recovered from COVID-19. 🙌🏼\n·\n·\nTogether, we can get through this. 💛\n·\n·\n·\n·\n·\n·\n·\n·\n·\n·\n·\n·\n·\n·\n·\n#travelingwithjessica #quarantine2020 #quarantinestories #stayhomestaysafe #strongertogether #thankshealthheroes #flattenthecurve #thenewnormal #quarantinediaries #wereinthistogether #plasmadonation #newyorkbloodcenter",125,-0.041827
1,1543,2019-05-14 00:00:00,baileydoodle,newyorkbloodcenter,Happy Therapy Dog Tuesday!! ❤️🐾🐶 I LOVE my job - I get to make people happy and in turn they make me so happy too! 😊 #baileyandcharlie #newyorkbloodcenter #blooddrive #giveblood #savealife #therapydog #pettherapy #thepowerthepaw #cohenchildrensmedicalcenter .\n.\n❤️BLOOD DRIVE UPDATE❤️: 196 blood donations were received yesterday!! Thank you to all who donated or who tried to donate!! ❤️,87,0.825000
2,899,2020-10-27 00:00:00,belinda_vegan,newyorkbloodcenter,"Research Chimpanzees Infected, Abandoned, Starved, and Rescued.\n_\nChimpanzees infected with hepatitis were dumped on Liberia's Monkey Island. The truth about the horrific lives of these sentient animal beings. Their treatment by a group of researchers from the New York Blood Center was inexcusable: they infected them with hepatitis and river blindness beginning in the 1970s. The researchers fled Liberia during a deadly Ebola outbreak and the abandoned chimpanzees were left to starve. They couldn't be released into the wild because they could spread the disease and didn't know how to forage for food. \n_\nJoseph Thomas, a previous caretaker at the researchers' laboratory, steadfastly remained. He came to their rescue and stayed with the chimpanzees when the researchers fled, despite their carrying the disease. He says he ""remembers pulling up to islands and seeing frantic, desperate animals. They screamed and fought over scraps. It wasn’t enough."" Thomas now leads a dedicated team of caretakers. Thomas is quoted saying: ""They hate to be cooped up. They laugh, cry, get jealous and have temper tantrums—'just like us.'\n_\nThomas said: “I’ll be doing this ... until they die or I do.”\n#belinda_vegan\nEMBRACE VEGANISM 🌱\nwww. Challenge 22.com\n_\n📽️ @rawtruth_ \n_\nYou have never really lived until you have done something for someone who can never repay you.💞\n_\n#understandlife #vegansofig #kindhearted #vegan #govegan #vegsnsofig #vegansofinstagram #monkey #chimp #animals #nature #animallover #animalsofinstagram #love #respect #giving #compassion #helpingothers #love #compassion 

#### 2.3: Is there a relationship between any engagment metrics and sentiment?

In [30]:
# categorize sentiment column based on value
def sentiment_label(sentiment):
    if sentiment > 0:
        return 'Positive'
    elif sentiment <0:
        return 'Negative'
    else:
        return 'Neutral'

In [31]:
#apply to instagram dataframe and check counts
mentions_ig['sentiment_label'] = mentions_ig['sentiment'].apply(sentiment_label)
mentions_ig.sentiment_label.value_counts()

Neutral     1086
Positive    1034
Negative    178 
Name: sentiment_label, dtype: int64

In [32]:
#do the same for Twitter
mentions_tw['sentiment_label'] = mentions_tw['sentiment'].apply(sentiment_label)
mentions_tw.sentiment_label.value_counts()

Neutral     9654
Positive    1169
Negative    491 
Name: sentiment_label, dtype: int64

#### Subquestion 2: Comparison of positive and negative hashtags associated with these organizations<br>


In [33]:
#first check number of Instagram posts mentioning each org
for i in list(mentions_ig.cited.unique()):
    print('Total number of Instagram posts mentioning {0}: {1}'.format(i,len(mentions_ig.loc[mentions_ig['cited'] == i])))

Total number of Instagram posts mentioning avisnazionale: 791
Total number of Instagram posts mentioning fidasdonatorisangue: 47
Total number of Instagram posts mentioning newyorkbloodcenter: 1460


In [34]:
#do the same for Twitter
for i in list(mentions_tw.cited.unique()):
    print('Total number of Twitter posts mentioning {0}: {1}'.format(i,len(mentions_tw.loc[mentions_tw['cited'] == i])))

Total number of Twitter posts mentioning avisnazionale: 9837
Total number of Twitter posts mentioning AVIS_Milano: 22
Total number of Twitter posts mentioning NYBloodCenter: 1283
Total number of Twitter posts mentioning FIDASnazionale: 142
Total number of Twitter posts mentioning nan: 0
Total number of Twitter posts mentioning RedCrossENY: 27


Are the posts mentioning these organizations mostly positive or negative?

In [35]:
#first check range of sentiment column for Instgram and Twitter
print(mentions_ig.sentiment.min(),mentions_ig.sentiment.max())
print(mentions_tw.sentiment.min(),mentions_tw.sentiment.max())

-0.8 1.0
-1.0 1.0


In [36]:
#do the same for Twitter
mentions_tw['sentiment_label'] = mentions_tw['sentiment'].apply(sentiment_label)
mentions_tw.sentiment_label.value_counts()

Neutral     9654
Positive    1169
Negative    491 
Name: sentiment_label, dtype: int64

In [37]:
#prep data to visualize with rawgraphs
sentiment_ig = (pd.pivot_table(mentions_ig, values='sentiment', index=['cited'],
                    columns=['sentiment_label'], aggfunc='count',fill_value=0)).reset_index()
#add column for platform
sentiment_ig['platform'] = 'Instagram'

sentiment_tw = (pd.pivot_table(mentions_tw, values='sentiment', index=['cited'],
                    columns=['sentiment_label'], aggfunc='count',fill_value=0)).reset_index()
#add column for platform
sentiment_tw['platform'] = 'Twitter'

In [38]:
#stack dataframes to calc percentages of each sentiment
sentiment_all = sentiment_ig.append(sentiment_tw)
sentiment_all = sentiment_all.rename(columns={'cited':'Org.','Negative':'neg_count','Neutral':'neu_count','Positive':'pos_count'})
sentiment_all['total'] = sentiment_all['neg_count'] + sentiment_all['neu_count'] + sentiment_all['pos_count']
sentiment_all['Positive'] = round(((sentiment_all['pos_count'] / sentiment_all['total'])*100),1)
sentiment_all['Negative'] = round(((sentiment_all['neg_count'] / sentiment_all['total'])*100),1)
sentiment_all['Neutral'] = round(((sentiment_all['neu_count'] / sentiment_all['total'])*100),1)
sentiment_all

sentiment_label,Org.,neg_count,neu_count,pos_count,platform,total,Positive,Negative,Neutral
0,avisnazionale,26,667,98,Instagram,791,12.4,3.3,84.3
1,fidasdonatorisangue,1,44,2,Instagram,47,4.3,2.1,93.6
2,newyorkbloodcenter,151,375,934,Instagram,1460,64.0,10.3,25.7
0,AVIS_Milano,0,21,1,Twitter,22,4.5,0.0,95.5
1,FIDASnazionale,1,138,3,Twitter,142,2.1,0.7,97.2
2,NYBloodCenter,304,266,713,Twitter,1283,55.6,23.7,20.7
3,RedCrossENY,1,9,17,Twitter,27,63.0,3.7,33.3
4,avisnazionale,185,9217,435,Twitter,9837,4.4,1.9,93.7


In [39]:
#standardize org names before exporting
sentiment_all['Org.'].unique()

array(['avisnazionale', 'fidasdonatorisangue', 'newyorkbloodcenter',
       'AVIS_Milano', 'FIDASnazionale', 'NYBloodCenter', 'RedCrossENY'],
      dtype=object)

In [40]:
sentiment_all = sentiment_all.replace('newyorkbloodcenter','NYBloodCenter')
sentiment_all = sentiment_all.replace('fidasdonatorisangue','FIDASnazionale') #confirm this is the same
sentiment_all['Org.'].unique()

array(['avisnazionale', 'FIDASnazionale', 'NYBloodCenter', 'AVIS_Milano',
       'RedCrossENY'], dtype=object)

In [41]:
#export csv to visualize in rawgraphs as stacked bar chart
sentiment_all.to_csv('data_rawgraphs/sentiment_all.csv')
#export another csv has orgs compared across twitter and platform (AVIS, FIDAS and NYBloodCenter)
sentiment_comparison = sentiment_all.loc[(sentiment_all['Org.'] == 'avisnazionale') |
                                          (sentiment_all['Org.'] == 'FIDASnazionale') |
                                          (sentiment_all['Org.'] == 'NYBloodCenter')]
sentiment_comparison.to_csv('data_rawgraphs/sentiment_comparison.csv')

In [42]:
#create dfs to plot monthly posts by and mentions of org
def monthly_groupby_sentiment(data):
    data['Number of Posts'] = 1
    data['date'] = pd.to_datetime(data['date'])
    data['year-month'] = data['date'].dt.strftime('%Y-%m')
    data = data.groupby(['year-month','username']).mean({'sentiment'}).reset_index(drop=False)
    return data

def monthly_groupby_mentions(data):
    data['Number of Mentions'] = 1
    data['date'] = pd.to_datetime(data['date'])
    data['year-month'] = data['date'].dt.strftime('%Y-%m')
    data = data.groupby(['year-month','cited']).agg({'Number of Mentions':sum}).reset_index(drop=False)
    return data

In [43]:
daily_tw = mentions_tw[['date','cited','sentiment']].groupby(['date','cited']).mean().reset_index()
daily_tw['platform'] = 'Twitter'
daily_ig = mentions_ig[['date','cited','sentiment']].groupby(['date','cited']).mean().reset_index()
daily_ig['platform'] = 'Instagram'
daily_mentions = daily_tw.append(daily_ig)
#make sure date is in proper format
daily_mentions['date'] = pd.to_datetime(daily_mentions['date'])

In [44]:
mentions_tw.cited.unique()

array(['avisnazionale', 'AVIS_Milano', 'NYBloodCenter', 'FIDASnazionale',
       nan, 'RedCrossENY'], dtype=object)

In [45]:
mentions_tw_edit = mentions_tw[['date','cited','tweet','sentiment']]
mentions_tw_edit.loc[mentions_tw_edit['cited'] == 'avisnazionale'].sort_values('sentiment',ascending=False).head(15)

,date,cited,tweet,sentiment
9101,2015-11-22,avisnazionale,@ValeZanatta @avisnazionale proud of you!!!,1.0
2727,2019-12-12,avisnazionale,"@GiorgiaMeloni @avisnazionale Dunque : dalla polizia sei stata, dei migranti parli, dei Rom e dei presepi , della patria ed ora come LVI , The best- ia , la foto mentre doni sangue seppur Rapa , ti mancano le felpe e la nutella e sei parodia perfetta.",1.0
4495,2018-11-20,avisnazionale,@AbzeroSrls @startup_italia @avisnazionale @ScuolaSantAnna @PoloNavacchio @JOINTTO_IT @AXAItalia @BioUpperItalia Good luck!! Great job!!,1.0
8092,2016-10-02,avisnazionale,"Great we were together at @Culturit event today, speaking about innovation experiences @avisnazionale, @Fondoambiente and #Alumnitaliane! https://t.co/FHJShEg9O4",1.0
6698,2017-11-26,avisnazionale,"@eli_valli @avisnazionale Piacere, piú che dovere, in questi casi :D",1.0
8438,2016-07-06,avisnazionale,Tomorrow @EMCItalia is proud to host @avisnazionale Donating is an act of solidarity We roll up our sleeves ! https://t.co/63wMlPpAZw,1.0
9364,2015-09-04,avisnazionale,@frap1975 @avisnazionale @avisterni Bravo amore mio! &lt;3 So proud of you!,1.0
10224,2021-02-08,avisnazionale,Blood donation today @avisnazionale I feel happy! #blooddonation #rossosangue,1.0
4522,2018-11-15,avisnazionale,#stefaniaantonioni presenta alcune best practice “emozionali” di #comunicazionesociale: il caso di @avisnazionale #Laprimavolta #lenuovefrontieredellacomunocazionesociale @AidoPU @geaducci https://t.co/F9D6TVAvkB,1.0
9413,2015-08-01,avisnazionale,"@avisnazionale @DonaConAmore: l'evento salernitano sulla #donazione del #sangue, Seguici! :D",1.0


In [46]:
mentions_tw.columns

Index(['Unnamed: 0', 'id', 'conversation_id', 'created_at', 'date', 'time',
       'timezone', 'user_id', 'username', 'name', 'place', 'tweet', 'language',
       'mentions', 'urls', 'photos', 'replies_count', 'retweets_count',
       'likes_count', 'hashtags', 'cashtags', 'link', 'retweet', 'quote_url',
       'video', 'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest', 'cited', 'sentiment', 'sentiment_label'],
      dtype='object')

In [47]:
#is there any correlation between sentiment score and other engagment metrics?
mentions_ig[['replies_count','likes_count','sentiment']].corr() 

,replies_count,likes_count,sentiment
replies_count,1.000000,0.497167,0.057905
likes_count,0.497167,1.000000,0.064398
sentiment,0.057905,0.064398,1.000000


Overall, there does not seem to be a relationship between sentiment and replies or likes count but break out by 
blood donation organization to confirm. 

In [50]:
orgs_ig = list(mentions_ig.cited.unique())
for i in orgs_ig:
    print(i,'- Instagram')
    print(mentions_ig.loc[mentions_ig['cited'] == i][['replies_count','likes_count','sentiment']].corr())
    print('-----------------------------------------------------')

avisnazionale - Instagram
               replies_count  likes_count  sentiment
replies_count  1.000000       0.806597     0.097251 
likes_count    0.806597       1.000000     0.142548 
sentiment      0.097251       0.142548     1.000000 
-----------------------------------------------------
fidasdonatorisangue - Instagram
               replies_count  likes_count  sentiment
replies_count  1.000000       0.555352     0.216650 
likes_count    0.555352       1.000000     0.247989 
sentiment      0.216650       0.247989     1.000000 
-----------------------------------------------------
newyorkbloodcenter - Instagram
               replies_count  likes_count  sentiment
replies_count  1.000000       0.417216     0.016936 
likes_count    0.417216       1.000000     0.030183 
sentiment      0.016936       0.030183     1.000000 
-----------------------------------------------------


In [51]:
#same for Twitter
mentions_tw[['replies_count','retweets_count','likes_count','sentiment']].corr()

,replies_count,retweets_count,likes_count,sentiment
replies_count,1.000000,0.530912,0.849744,0.004629
retweets_count,0.530912,1.000000,0.687572,-0.005089
likes_count,0.849744,0.687572,1.000000,0.001630
sentiment,0.004629,-0.005089,0.001630,1.000000


In [52]:
#Overall, there doesn't seem to be any correlation with sentiment and other engagment metrics, break out by org to see if any different
orgs_tw = list(mentions_tw.cited.unique())
for i in orgs_tw:
    print(i,'- Twitter')
    print(mentions_tw.loc[mentions_tw['cited'] == i][['replies_count','retweets_count','likes_count','sentiment']].corr())
    print('--------------------------------------------------------------------')

avisnazionale - Twitter
                replies_count  retweets_count  likes_count  sentiment
replies_count   1.000000       0.536493        0.853443    -0.000915 
retweets_count  0.536493       1.000000        0.694851    -0.005493 
likes_count     0.853443       0.694851        1.000000    -0.004489 
sentiment      -0.000915      -0.005493       -0.004489     1.000000 
--------------------------------------------------------------------
AVIS_Milano - Twitter
                replies_count  retweets_count  likes_count  sentiment
replies_count   1.000000      -0.191667       -0.200568    -0.069007 
retweets_count -0.191667       1.000000        0.657417    -0.195519 
likes_count    -0.200568       0.657417        1.000000    -0.216480 
sentiment      -0.069007      -0.195519       -0.216480     1.000000 
--------------------------------------------------------------------
NYBloodCenter - Twitter
                replies_count  retweets_count  likes_count  sentiment
replies_count   1.0000

Overall, there doesn't seem to be any correlation with sentiment and other engagment metrics on Instagram and Twitter.